##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Estimator

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

> 警告: 新しいコードには Estimators は推奨されません。Estimators は `v1.Session` スタイルのコードを実行しますが、これは正しく記述するのはより難しく、特に TF 2 コードと組み合わせると予期しない動作をする可能性があります。Estimators は、[互換性保証](https://tensorflow.org/guide/versions)の対象となりますが、セキュリティの脆弱性以外の修正は行われません。詳細については、[移行ガイド](https://tensorflow.org/guide/migrate)を参照してください。

このドキュメントでは、`tf.estimator` という高位 TensorFlow API を紹介します。Estimator は以下のアクションをカプセル化します。

- トレーニング
- 評価
- 予測
- 配信向けエクスポート

TensorFlow は、事前に作成された複数の Estimator を実装します。カスタムの Estimator は依然としてサポートされていますが、主に下位互換性の対策としてサポートされているため、**新しいコードでは、カスタム Estimator を使用してはいけません**。事前に作成された Estimator とカスタム Estimator はすべて、`tf.estimator.Estimator` クラスに基づくクラスです。

簡単な例については、[Estimator チュートリアル](../tutorials/estimator/linear.ipynb)を試してください。API デザインの概要については、[ホワイトペーパー](https://arxiv.org/abs/1708.02637)をご覧ください。

## セットアップ

In [ ]:
!pip install -U tensorflow_datasets

In [ ]:
import tempfile
import os

import tensorflow as tf
import tensorflow_datasets as tfds

## メリット

`tf.keras.Model` と同様に、`estimator` はモデルレベルの抽象です。`tf.estimator` は、`tf.keras` 向けに現在開発段階にある以下の機能を提供しています。

- パラメーターサーバーベースのトレーニング
- [TFX](http://tensorflow.org/tfx) の完全統合

## Estimator の機能

Estimator には以下のメリットがあります。

- Estimator ベースのモデルは、モデルを変更することなくローカルホストまたは分散マルチサーバー環境で実行できます。さらに、モデルをコーディングし直すことなく、CPU、GPU、または TPU で実行できます。
- Estimator では、次を実行する方法とタイミングを制御する安全な分散型トレーニングループを使用できます。
    - データの読み込み
    - 例外の処理
    - チェックポイントファイルの作成と障害からの復旧
    - TensorBoard 用のサマリーの保存

Estimator を使ってアプリケーションを記述する場合、データ入力パイプラインとモデルを分離する必要があります。分離することで、異なるデータセットを伴う実験を単純化することができます。

## 事前作成済み Estimator を使用する

既成の Estimator を使うと、基本の TensorFlow API より非常に高い概念レベルで作業することができます。Estimator がすべての「配管作業」を処理してくれるため、計算グラフやセッションの作成などに気を回す必要がありません。さらに、事前作成済みの Estimator では、コード変更を最小限に抑えて多様なモデルアーキテクチャを使った実験を行えます。たとえば `tf.estimator.DNNClassifier` は、密度の高いフィードフォワードのニューラルネットワークに基づく分類モデルをトレーニングする事前作成済みの Estimator クラスです。

事前作成済み Estimator に依存する TensorFlow プログラムは、通常、次の 4 つのステップで構成されています。

### 1. 入力関数を作成する

たとえば、トレーニングセットをインポートする関数とテストセットをインポートする関数を作成する場合、Estimator は入力が次の 2 つのオブジェクトのペアとしてフォーマットされていることを期待します。

- 特徴名のキーと対応する特徴データを含むテンソル（または SparseTensors）の値で構成されるディクショナリ
- 1 つ以上のラベルを含むテンソル

`input_fn` は上記のフォーマットのペアを生成する `tf.data.Dataset` を返します。

たとえば、次のコードは Titanic データセットの `train.csv` ファイルから `tf.data.Dataset` を構築します。

In [ ]:
def train_input_fn():
  titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
  titanic = tf.data.experimental.make_csv_dataset(
      titanic_file, batch_size=32,
      label_name="survived")
  titanic_batches = (
      titanic.cache().repeat().shuffle(500)
      .prefetch(tf.data.AUTOTUNE))
  return titanic_batches

`input_fn` は、`tf.Graph` で実行し、グラフテンソルを含む `(features_dics, labels)` ペアを直接返すこともできますが、定数を返すといった単純なケースではない場合に、エラーが発生しやすくなります。

### 2. 特徴量カラムを定義する

`tf.feature_column` は、特徴量名、その型、およびすべての入力前処理を特定します。

たとえば、次のスニペットは 3 つの特徴量カラムを作成します。

- 最初の特徴量カラムは、浮動小数点数の入力として直接 `age` 特徴量を使用します。
- 2 つ目の特徴量カラムは、カテゴリカル入力として `class` 特徴量を使用します。
- 3 つ目の特徴量カラムは、カテゴリカル入力として `embark_town` を使用しますが、オプションを列挙する必要がないように、またオプション数を設定するために、`hashing trick` を使用します。

詳細については、[特徴量カラムのチュートリアル](https://www.tensorflow.org/tutorials/keras/feature_columns)をご覧ください。

In [ ]:
age = tf.feature_column.numeric_column('age')
cls = tf.feature_column.categorical_column_with_vocabulary_list('class', ['First', 'Second', 'Third']) 
embark = tf.feature_column.categorical_column_with_hash_bucket('embark_town', 32)

### 3. 関連する事前作成済み Estimator をインスタンス化する

`LinearClassifier` という事前作成済み Estimator のインスタンス化の例を次に示します。

In [ ]:
model_dir = tempfile.mkdtemp()
model = tf.estimator.LinearClassifier(
    model_dir=model_dir,
    feature_columns=[embark, cls, age],
    n_classes=2
)

詳細については、[線形分類器のチュートリアル](https://www.tensorflow.org/tutorials/estimator/linear)をご覧ください。

### 4. トレーニング、評価、または推論メソッドを呼び出す

すべての Estimator には、`train`、`evaluate`、および `predict` メソッドがあります。


In [ ]:
model = model.train(input_fn=train_input_fn, steps=100)

In [ ]:
result = model.evaluate(train_input_fn, steps=10)

for key, value in result.items():
  print(key, ":", value)

In [ ]:
for pred in model.predict(train_input_fn):
  for key, value in pred.items():
    print(key, ":", value)
  break

### 事前作成済み Estimator のメリット

事前作成済み Estimator は、次のようなベストプラクティスをエンコードするため、さまざまなメリットがあります。

- さまざまな部分の計算グラフをどこで実行するかを決定し、単一のマシンまたはクラスタに戦略を実装するためのベストプラクティス。
- イベント（要約）の書き込みと普遍的に役立つ要約のベストプラクティス。

事前作成済み Estimator を使用しない場合は、上記の特徴量を独自に実装する必要があります。

## カスタム Estimator

事前作成済みかカスタムかに関係なく、すべての Estimator の中核は、*モデル関数*の `model_fn` にあります。これは、トレーニング、評価、および予測に使用するグラフを構築するメソッドです。事前作成済み Estimator を使用する場合は、モデル関数はすでに実装されていますが、カスタム Estimator を使用する場合は、モデル関数を自分で記述する必要があります。

> 注意: カスタム `model_fn` は 1.x スタイルのグラフモードでそのまま実行します。つまり、Eager execution はなく、依存関係の自動制御もないため、`tf.estimator` からカスタム `model_fn` に移行する必要があります。代替の API は `tf.keras` と `tf.distribute` です。トレーニングの一部に `Estimator` を使用する必要がある場合は、`tf.keras.estimator.model_to_estimator` コンバータを使用して `keras.Model` から `Estimator` を作成する必要があります。

## Keras モデルから Estimator を作成する

`tf.keras.estimator.model_to_estimator` を使用して、既存の Keras モデルを Estimator に変換できます。モデルコードを最新の状態に変更したくても、トレーニングパイプラインに Estimator が必要な場合に役立ちます。

Keras MobileNet V2 モデルをインスタンス化し、トレーニングに使用する optimizer、loss、および metrics とともにモデルをコンパイルします。

In [ ]:
keras_mobilenet_v2 = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3), include_top=False)
keras_mobilenet_v2.trainable = False

estimator_model = tf.keras.Sequential([
    keras_mobilenet_v2,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])

# Compile the model
estimator_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

コンパイルされた Keras モデルから `Estimator` を作成します。Keras モデルの初期化状態が、作成した `Estimator` に維持されます。

In [ ]:
est_mobilenet_v2 = tf.keras.estimator.model_to_estimator(keras_model=estimator_model)

派生した `Estimator` をほかの `Estimator` と同じように扱います。

In [ ]:
IMG_SIZE = 160  # All images will be resized to 160x160

def preprocess(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [ ]:
def train_input_fn(batch_size):
  data = tfds.load('cats_vs_dogs', as_supervised=True)
  train_data = data['train']
  train_data = train_data.map(preprocess).shuffle(500).batch(batch_size)
  return train_data

トレーニングするには、Estimator の train 関数を呼び出します。

In [ ]:
est_mobilenet_v2.train(input_fn=lambda: train_input_fn(32), steps=50)

同様に、評価するには、Estimator の evaluate 関数を呼び出します。

In [ ]:
est_mobilenet_v2.evaluate(input_fn=lambda: train_input_fn(32), steps=10)

詳細については、`tf.keras.estimator.model_to_estimator` のドキュメントを参照してください。

## Estimator でオブジェクトベースのチェックポイントを保存する

Estimator はデフォルトで、[チェックポイントガイド](checkpoint.ipynb)で説明したオブジェクトグラフではなく、変数名でチェックポイントを保存します。`tf.train.Checkpoint` は名前ベースのチェックポイントを読み取りますが、モデルの一部を Estimator の `model_fn` の外側に移動すると変数名が変わることがあります。上位互換性においては、オブジェクトベースのチェックポイントを保存すると、Estimator の内側でモデルをトレーニングし、外側でそれを使用することが容易になります。

In [ ]:
import tensorflow.compat.v1 as tf_compat

In [ ]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [ ]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [ ]:
def model_fn(features, labels, mode):
  net = Net()
  opt = tf.keras.optimizers.Adam(0.1)
  ckpt = tf.train.Checkpoint(step=tf_compat.train.get_global_step(),
                             optimizer=opt, net=net)
  with tf.GradientTape() as tape:
    output = net(features['x'])
    loss = tf.reduce_mean(tf.abs(output - features['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  return tf.estimator.EstimatorSpec(
    mode,
    loss=loss,
    train_op=tf.group(opt.apply_gradients(zip(gradients, variables)),
                      ckpt.step.assign_add(1)),
    # Tell the Estimator to save "ckpt" in an object-based format.
    scaffold=tf_compat.train.Scaffold(saver=ckpt))

tf.keras.backend.clear_session()
est = tf.estimator.Estimator(model_fn, './tf_estimator_example/')
est.train(toy_dataset, steps=10)

その後、`tf.train.Checkpoint` は Estimator のチェックポイントをその `model_dir` から読み込むことができます。

In [ ]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(
  step=tf.Variable(1, dtype=tf.int64), optimizer=opt, net=net)
ckpt.restore(tf.train.latest_checkpoint('./tf_estimator_example/'))
ckpt.step.numpy()  # From est.train(..., steps=10)

## Estimator の SavedModel

Estimator は、[`tf.Estimator.export_saved_model`](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator#export_saved_model) によって SavedModel をエクスポートします。

In [ ]:
input_column = tf.feature_column.numeric_column("x")

estimator = tf.estimator.LinearClassifier(feature_columns=[input_column])

def input_fn():
  return tf.data.Dataset.from_tensor_slices(
    ({"x": [1., 2., 3., 4.]}, [1, 1, 0, 0])).repeat(200).shuffle(64).batch(16)
estimator.train(input_fn)

`Estimator` を保存するには、`serving_input_receiver` を作成する必要があります。この関数は、SavedModel が受け取る生データを解析する `tf.Graph` の一部を構築します。

`tf.estimator.export` モジュールには、これらの `receivers` を構築するための関数が含まれています。


次のコードは、`feature_columns` に基づき、[tf-serving](https://tensorflow.org/serving) と合わせて使用されることの多いシリアル化された `tf.Example` プロトコルバッファを受け入れるレシーバーを構築します。

In [ ]:
tmpdir = tempfile.mkdtemp()

serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec([input_column]))

estimator_base_path = os.path.join(tmpdir, 'from_estimator')
estimator_path = estimator.export_saved_model(estimator_base_path, serving_input_fn)

また、Python からモデルを読み込んで実行することも可能です。

In [ ]:
imported = tf.saved_model.load(estimator_path)

def predict(x):
  example = tf.train.Example()
  example.features.feature["x"].float_list.value.extend([x])
  return imported.signatures["predict"](
    examples=tf.constant([example.SerializeToString()]))

In [ ]:
print(predict(1.5))
print(predict(3.5))

`tf.estimator.export.build_raw_serving_input_receiver_fn` を使用すると、`tf.train.Example` の代わりに生のテンソルを取る入力関数を作成することができます。

## Estimator を使った `tf.distribute.Strategy` の使用（制限サポート）

`tf.estimator` は、もともと非同期パラメーターサーバー手法をサポートしていた分散型トレーニング TensorFlow API です。`tf.estimator` は現在では `tf.distribute.Strategy` をサポートするようになっています。`tf.estimator` を使用している場合は、コードを少し変更するだけで、分散型トレーニングに変更することができます。これにより、Estimator ユーザーは複数の GPU と複数のワーカーだけでなく、TPU でも同期分散型トレーニングを実行できるようになりましたが、Estimator でのこのサポートには制限があります。詳細については、以下に示す「[現在、何がサポートされていますか](#estimator_support)」セクションをご覧ください。

Estimator での `tf.distribute.Strategy` の使用は、Keras の事例とわずかに異なります。`strategy.scope` を使用する代わりに、ストラテジーオブジェクトを Estimator の `RunConfig` に渡します。

詳細については、[分散型トレーニングガイド](distributed_training.ipynb)をご覧ください。

次は、事前に作成された Estimator `LinearRegressor` と `MirroredStrategy` を使ってこの動作を示すコードスニペットです。


In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(
    train_distribute=mirrored_strategy, eval_distribute=mirrored_strategy)
regressor = tf.estimator.LinearRegressor(
    feature_columns=[tf.feature_column.numeric_column('feats')],
    optimizer='SGD',
    config=config)

ここでは、事前に作成された Estimator が使用されていますが、同じコードはカスタム Estimator でも動作します。`train_distribute` はトレーニングの分散方法を判定し、`eval_distribute` は評価の分散方法を判定します。この点も、トレーニングと評価に同じストラテジーを使用する Keras と異なるところです。

入力関数を使用して、この Estimator をトレーニングし、評価することができます。


In [ ]:
def input_fn():
  dataset = tf.data.Dataset.from_tensors(({"feats":[1.]}, [1.]))
  return dataset.repeat(1000).batch(10)
regressor.train(input_fn=input_fn, steps=10)
regressor.evaluate(input_fn=input_fn, steps=10)

Estimator と Keras のもう 1 つの違いとして強調すべき点は入力の処理方法です。Keras では、データセットの各バッチは複数のレプリカに自動的に分断されますが、Estimator の場合は、バッチの自動分断やワーカーをまたいで自動的にシャーディングすることもありません。ワーカーやデバイスでのデータの分散方法はユーザーが完全に制御するものであるため、`input_fn` を提供してデータの分散方法を指定する必要があります。

`input_fn` はワーカー当たり一度呼び出されるため、ワーカー当たり 1 つのデータセットが与えられます。次に、そのデータセットの 1 つのバッチがそのワーカーの 1 つのレプリカに供給され、したがって、1 つのワーカーの N 個のレプリカに対して N 個のバッチが消費されることになります。言い換えると、`input_fn` が返すデータセットは、サイズ `PER_REPLICA_BATCH_SIZE` のバッチを提供するということです。ステップのグローバルバッチサイズは、`PER_REPLICA_BATCH_SIZE * strategy.num_replicas_in_sync` として取得することができます。

マルチワーカートレーニングを行う場合は、データをワーカー間で分割するか、それぞれにランダムシードを使用してシャッフルする必要があります。これを行う方法の例は、「[Estimator を使ったマルチワーカートレーニング](../tutorials/distribute/multi_worker_with_estimator.ipynb)」を参照してください。

また、同様に、マルチワーカーとパラメーターサーバーストラテジーを使用することができます。コードは変わりませんが、`tf.estimator.train_and_evaluate` を使用し、クラスタで実行している各バイナリの `TF_CONFIG` 環境変数を設定する必要があります。

<a name="estimator_support"></a>

### 現在、何がサポートされていますか？

`TPUStrategy` を除くすべてのストラテジーを使った Estimator でのトレーニングのサポートには制限があります。基本的なトレーニングと評価は機能しますが、`v1.train.Scaffold` などの多数の高度な機能はまだ機能しません。また、この統合には多数のバグも存在する可能性があります。現時点では、Keras とカスタムトレーニングループのサポートに注力しているため、このサポートを積極的に改善する予定はありません。可能な限り、それらの API で `tf.distribute` を使用するようにしてください。

トレーニング API | MirroredStrategy | TPUStrategy | MultiWorkerMirroredStrategy | CentralStorageStrategy | ParameterServerStrategy
:-- | :-- | :-- | :-- | :-- | :--
Estimator API | 制限サポート | 未サポート | 制限サポート | 制限サポート | 制限サポート

### 例とチュートリアル

次は、Estimator によるさまざまなストラテジーの使用方法を示す、エンドツーエンドの例です。

1. [Estimator を使ったマルチワーカートレーニングのチュートリアル](../tutorials/distribute/multi_worker_with_estimator.ipynb)には、MNIST データセットで `MultiWorkerMirroredStrategy` を使って複数のワーカーをトレーニングする方法が説明されています。
2. Kubernetes テンプレートを使った `tensorflow/ecosystem` で[分散ストラテジーによってマルチワーカートレーニングを実行する](https://github.com/tensorflow/ecosystem/tree/master/distribution_strategy)エンドツーエンドの例。Keras モデルから始め、`tf.keras.estimator.model_to_estimator` API を使って Estimator に変換します。
3. [ResNet50](https://github.com/tensorflow/models/blob/master/official/vision/image_classification/resnet_imagenet_main.py) の公式モデル。`MirroredStrategy` または `MultiWorkerMirroredStrategy` を使ってトレーニングできます。